In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras import datasets, models, layers
import matplotlib.pyplot as plt

In [1]:
img_height = 28
img_weight = 28
batch_size = 2

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d emisssg/emotions
!unzip emotions.zip

In [7]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/images",
    labels = "inferred",
    label_mode = "int",
    color_mode = "grayscale",
    batch_size = batch_size,
    image_size = (img_height, img_weight),
    shuffle = True,
    seed = 200,
    validation_split = 0.1,
    subset = "training",

)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/images",
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_weight),
    shuffle=True,
    seed=200,
    validation_split=0.1,
    subset="validation",

)

Found 35887 files belonging to 2 classes.
Using 32299 files for training.
Found 35887 files belonging to 2 classes.
Using 3588 files for validation.


In [8]:
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y

ds_train = ds_train.map(augment)

In [22]:
class CNNBlock(layers.Layer):
  def __init__(self, input_tensors, kernel_size = 3):
    super(CNNBlock, self).__init__()
    self.conv1 = layers.Conv2D(input_tensors, kernel_size, padding = "same")
    self.bn = layers.BatchNormalization()

  def call(self, out_tensors, training = False):
    x = self.conv1(out_tensors)
    x = self.bn(x, training = training)
    return tf.nn.relu(x)

In [23]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x

In [24]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=6):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

In [25]:
model = ResNet_Like()

In [26]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"],
)

In [27]:
model.fit(ds_train, batch_size = 128, epochs = 1, verbose = 1)

16150/16150 [==============================] - 135s 8ms/step - loss: 0.5188 - accuracy: 0.7987


In [28]:
model.evaluate(
    ds_validation, batch_size = 64
)

1794/1794 [==============================] - 8s 4ms/step - loss: 0.4954 - accuracy: 0.8041


[0.49542340636253357, 0.8040691018104553]

In [29]:
model.summary()

Model: "res_net__like"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block (ResBlock)        multiple                  28896     
                                                                 
 res_block_1 (ResBlock)      multiple                  592512    
                                                                 
 res_block_2 (ResBlock)      multiple                  2364032   
                                                                 
 global_average_pooling2d (G  multiple                 0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               multiple                  3078      
                                                                 
Total params: 2,988,518
Trainable params: 2,985,446
Non-trainable params: 3,072
_______________________________________

In [33]:
model.save_weights("/content/weight")